# Task 7 IBM Injection/Extraction Timing Demo

This notebook focuses only on **how Task 7 data is injected** and **when the readout is extracted**.
It mirrors the Task 7 settings used in `example_tasks/task_7_learn_logic16_ibm.py`.


## Imports


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from computingMicrobiome.benchmarks.k_opcode_logic16_bm import (
    N_CHANNELS,
    TAG_OP0,
    TAG_OP1,
    TAG_OP2,
    TAG_OP3,
    TAG_A,
    TAG_B,
    VAL_0,
    VAL_1,
    DIST,
    CUE,
    build_tagged_stream,
    run_episode_record_tagged,
)
from computingMicrobiome.ibm import make_ibm_config_from_species
from computingMicrobiome.utils import create_input_locations


## Task 7 Configuration (same as script)


In [ ]:
# Core task settings
BOUNDARY = "periodic"
RECURRENCE_TARGET = 8
ITR = 8
D_PERIOD = 200
REPEATS = 1
FEATURE_MODE = "cue_tick"
OUTPUT_WINDOW = 2
SEED_TRAIN = 0

# IBM config used by Task 7
LOGIC16_STREAM_LEN = (REPEATS * 6) + D_PERIOD + 1
TRACE_DEPTH = LOGIC16_STREAM_LEN * (ITR + 1) + 8

IBM_CFG = make_ibm_config_from_species(
    species_indices=[0, 1, 2],
    height=8,
    width_grid=8,
    overrides={
        "state_width_mode": "raw",
        "input_trace_depth": TRACE_DEPTH,
        "input_trace_channels": N_CHANNELS,
        "input_trace_decay": 1.0,
        "inject_scale": 0.0,
        "dilution_p": 0.0,
        "diff_numer": 0,
    },
)

WIDTH = int(IBM_CFG["height"]) * int(IBM_CFG["width_grid"])
RECURRENCE = min(RECURRENCE_TARGET, max(1, WIDTH // N_CHANNELS))
ITER_BETWEEN = ITR + 1

print("WIDTH:", WIDTH)
print("N_CHANNELS:", N_CHANNELS)
print("RECURRENCE:", RECURRENCE)
print("ITER_BETWEEN (sim steps per tick):", ITER_BETWEEN)
print("TRACE_DEPTH:", TRACE_DEPTH)


## Build One Example Logic16 Input Stream

Example sample:
- opcode bits `[op3, op2, op1, op0] = [0, 1, 1, 0]`
- operands `a=1`, `b=0`


In [ ]:
op_bits = np.array([0, 1, 1, 0], dtype=np.int8)
a, b = 1, 0

input_streams = build_tagged_stream(
    op_bits_msb_first=op_bits,
    a=a,
    b=b,
    d_period=D_PERIOD,
    repeats=REPEATS,
    order=None,
)

L = input_streams.shape[0]
cue_tick = L - 1
extract_tick = cue_tick  # feature_mode='cue_tick' extracts the final tick feature
extract_step = extract_tick * ITER_BETWEEN

print("stream length L (ticks):", L)
print("cue tick index:", cue_tick)
print("readout extraction tick index:", extract_tick)
print("readout extraction simulation step:", extract_step)


## Visualize Tick-by-Tick Input Injection Pattern

Rows are channels and columns are task ticks.


In [ ]:
CHANNEL_NAMES = [
    "TAG_OP0", "TAG_OP1", "TAG_OP2", "TAG_OP3", "TAG_A", "TAG_B",
    "VAL_0", "VAL_1", "DIST", "CUE",
]

fig, ax = plt.subplots(figsize=(14, 4.8))
ax.imshow(input_streams.T, aspect="auto", interpolation="nearest", cmap="Greys")
ax.set_title("Task 7 input_streams (channels x ticks)")
ax.set_xlabel("tick index")
ax.set_ylabel("channel")
ax.set_yticks(np.arange(N_CHANNELS))
ax.set_yticklabels(CHANNEL_NAMES)

# Cell separators for easier visual inspection.
ax.set_xticks(np.arange(-0.5, input_streams.shape[0], 1), minor=True)
ax.set_yticks(np.arange(-0.5, N_CHANNELS, 1), minor=True)
ax.grid(which="minor", color="white", linewidth=0.6)
ax.tick_params(which="minor", bottom=False, left=False)

ax.axvline(cue_tick, color="red", linestyle="--", linewidth=2, label=f"cue/extract tick={cue_tick}")
ax.legend(loc="upper right")
plt.tight_layout()
plt.show()


## Tick Table: What Happens at Each Tick and Step


In [ ]:
def decode_tick(row: np.ndarray) -> tuple[str, int, str]:
    tag_indices = [TAG_OP0, TAG_OP1, TAG_OP2, TAG_OP3, TAG_A, TAG_B]
    tag_names = ["op0", "op1", "op2", "op3", "a", "b"]

    active_tag = None
    for idx, name in zip(tag_indices, tag_names):
        if row[idx] == 1:
            active_tag = name
            break

    val = None
    if row[VAL_0] == 1:
        val = 0
    elif row[VAL_1] == 1:
        val = 1

    if row[CUE] == 1:
        phase = "cue"
    elif active_tag is not None:
        phase = "write"
    else:
        phase = "distractor"

    return phase, val, active_tag if active_tag is not None else "-"

rows = []
for tick in range(L):
    phase, val, field = decode_tick(input_streams[tick])
    rows.append((tick, tick * ITER_BETWEEN, phase, field, val))

print("tick | sim_step | phase      | field | value")
print("-" * 52)
for tick, step, phase, field, val in rows[:12]:
    print(f"{tick:4d} | {step:8d} | {phase:10s} | {field:5s} | {str(val):>5s}")
print("...")
for tick, step, phase, field, val in rows[-8:]:
    print(f"{tick:4d} | {step:8d} | {phase:10s} | {field:5s} | {str(val):>5s}")


## Delay From Each Write Tick to Extraction

Extraction happens at the cue tick feature (`X_tick[-1]`), i.e., at `tick = L-1`.


In [ ]:
write_rows = []
for tick in range(L):
    phase, val, field = decode_tick(input_streams[tick])
    if phase == "write":
        delay_ticks = extract_tick - tick
        delay_steps = delay_ticks * ITER_BETWEEN
        write_rows.append((field, tick, extract_tick, delay_ticks, delay_steps, val))

print("field | write_tick | extract_tick | delay_ticks | delay_steps | written_value")
print("-" * 80)
for field, wt, et, dt, ds, val in write_rows:
    print(f"{field:5s} | {wt:10d} | {et:12d} | {dt:11d} | {ds:11d} | {val:13d}")


## Where Inputs Are Injected on the IBM Lattice

Each task tick injects each channel value at fixed channel-specific locations.
Those locations are sampled once from `create_input_locations(width, recurrence, N_CHANNELS, rng)`.


In [ ]:
rng = np.random.default_rng(SEED_TRAIN)
input_locations = create_input_locations(WIDTH, RECURRENCE, N_CHANNELS, rng)
channel_idx = np.arange(input_locations.size) % N_CHANNELS

print("locations per channel:")
for ch in range(N_CHANNELS):
    locs = input_locations[channel_idx == ch]
    print(f"  {CHANNEL_NAMES[ch]:8s} -> {locs.tolist()}")


## Verify Extraction Point Using the Real Episode Runner


In [ ]:
ep = run_episode_record_tagged(
    op_bits_msb_first=op_bits,
    a=a,
    b=b,
    rule_number=110,  # ignored by IBM backend, but required by API
    width=WIDTH,
    boundary=BOUNDARY,
    itr=ITR,
    d_period=D_PERIOD,
    rng=np.random.default_rng(SEED_TRAIN),
    input_locations=input_locations,
    repeats=REPEATS,
    order=None,
    reg=None,
    collect_states=True,
    x0_mode="zeros",
    feature_mode=FEATURE_MODE,
    output_window=OUTPUT_WINDOW,
    reservoir_kind="ibm",
    reservoir_config=IBM_CFG,
)

print("ep['L']:", ep["L"], "(ticks)")
print("ep['iter_between']:", ep["iter_between"], "(sim steps per tick)")
print("ep['T']:", ep["T"], "(total sim steps)")
print("ep['X_tick'].shape:", ep["X_tick"].shape)
print("ep['X_episode'].shape:", np.asarray(ep["X_episode"]).shape)

real_extract_tick = ep["L"] - 1
real_extract_step = real_extract_tick * ep["iter_between"]
print("real extraction tick:", real_extract_tick)
print("real extraction simulation step:", real_extract_step)


## Interpretation Checklist

Use this notebook to inspect:
1. Whether the write packets are where you expect.
2. Whether distractor duration (`D_PERIOD`) is too long for your setup.
3. Whether extraction at cue tick (`X_tick[-1]`) is the right readout point.
4. Whether recurrence/locations look reasonable for `WIDTH=64` and `N_CHANNELS=10`.
